In [25]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import sparse

# Defining the constants required
Z = 1 # hydrogren atom
dr = 0.2 # step size
N = 200 # number of grid points
r_vec = np.linspace(dr, N*dr, N) # radial grid
l = 0 # orbital quantum number

# Potential vector for Hydrogen
V_vec = -Z/r_vec


# Imaginary time step (Δτ)
delta = -1
E_k = -0.5
delta_tau = 2*delta/E_k
imagDeltaTau = (-1j*delta_tau)

In [26]:
# generates the sparse matrix where 0's are not stored in memory
# needs conversion into general matrix for proper viewing
def generateSparseMatrix(upper, mid, lower):
    return sparse.diags([lower, mid, upper], offsets=[-1, 0, 1], format="csr")

# fn to define the various matrices involved in the computation process
def create_matrices(r_vec, V_vec, Z, l, delta_tau):
    h = r_vec[1] - r_vec[0] # = 0.2
    N = len(r_vec) # legth of radial grid
    
    # creating the matrix D by defining the diagonals of the tridiagonal matrix
    D_lower = np.ones(N-1, dtype=complex) 
    D_upper = np.ones(N-1, dtype=complex) 
    D_mid = -2*np.ones(N, dtype=complex) 
    D = (1/h**2)*generateSparseMatrix(D_upper, D_mid, D_lower) # SPARSE, dense conversion not done (Only for viewing)
    
    # creating the matrix M2 in the same way
    M2_lower = np.ones(N-1, dtype=complex) 
    M2_upper = np.ones(N-1, dtype=complex) 
    M2_mid = 10*np.ones(N, dtype=complex)
    M2 = (-1/6)*generateSparseMatrix(M2_upper, M2_mid, M2_lower) # SPARSE
    
    # generating the diagonal potential matrix filled with the V_vec, SPARSE
    V_matrix = sparse.spdiags([V_vec], [0], N, N)
    
    # Effective hamiltonian for the CN step : D + M2.V (V=Vl, as l=0 here)
    H_eff= D + M2 @ V_matrix
    
